In [2]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from bs4 import Comment

In [6]:
search_results = []

In [9]:
for x in range(2, 45):
    try: 
        url = 'https://www.asial.com.au/members-directory/search/?command=getresults&pageNo={0}&LocationRadius=20&Filter::ServicesOffered::SelectMany=Alarm+Monitoring+Services'.format(x)
#         print(url)
        r = urllib.request.urlopen(url)
        soup = BeautifulSoup(r, 'html.parser')
        search_results += soup.find_all('div', class_="searchResults")
    except: 
        print("Damn, an error")
        pass

# Luckily for us, the designer wrapped the result in a conveniently named Class "searchResults" 

http://stackoverflow.com/questions/3507283/how-can-i-strip-comment-tags-from-html-using-beautifulsoup

This explains how to parse out comment tags

In [10]:
len(search_results)

846

In [11]:
companies = []

for result in search_results: 
# result = search_results[350]
    company = {}
    try: 
        company["title"] = result.find_all('h3')[0].get_text()
        company['profile'] = result.find('a')['href']

    #     This is an ugly dirty nasty hack to get 
        details = result.find('p').get_text()
        for entry in [x for x in details.split('\n') if x]: 
            if "Phone" in entry: 
                company['phone'] = entry.split(':')[1].strip()
            if 'Website' in entry: 
                company["website"] = entry.split(':')[1].strip()

        # This is an interesting one because it is hidden as a comment. I feed it back into BeautifulSoup
        text = result.find(text=lambda text:isinstance(text, Comment))
        company['email'] = BeautifulSoup(text, 'html.parser').find('a').get_text()
#         print(company)
        companies.append(company)
    except: 
        pass

In [12]:
len(companies)

825

In [13]:
db = pd.DataFrame(companies)
db2 = db.drop_duplicates('email')
db2.to_csv('../aus.csv', index=False)